# ReAct Agent with Query Engine Tools

## Build Query Engine Tools

In [10]:
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.tools import QueryEngineTool, ToolMetadata

In [2]:
try:
    storage_context = StorageContext.from_defaults(persist_dir="./storage/lyft")
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(persist_dir="./storage/uber")
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [3]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["../data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["../data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [4]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [5]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description="Provides information about Lyft financials for year 2021. "
            "Use a detailed plain text question as input to the tool.",
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description="Provides information about Uber financials for year 2021. "
            "Use a detailed plain text question as input to the tool.",
        ),
    ),
]

## Setup ReAct Agent

In [11]:
from llama_index.agent import ReActAgent
from llama_index.llms import OpenAI

In [12]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

In [13]:
response = agent.chat("What was Lyft's revenue growth in 2021?")
print(str(response))

Thought: I need to use a tool to help me answer the question.
Action: lyft_10k
Action Input: {'input': "What was Lyft's revenue growth in 2021?"}
Observation: 
Lyft's revenue growth in 2021 was 36%.
Response: Lyft's revenue growth in 2021 was 36%.
Lyft's revenue growth in 2021 was 36%.


In [14]:
response = agent.chat(
    "Compare and contrast the revenue growth of Uber and Lyft in 2021, then give an analysis"
)
print(str(response))

Thought: I need to use the lyft_10k and uber_10k tools to compare and contrast the revenue growth of Uber and Lyft in 2021.
Action: lyft_10k
Action Input: {'input': 'revenue growth in 2021'}
Observation: 
Revenue increased by 36% in 2021 as compared to the prior year.
Thought: I have obtained information about Lyft's revenue growth in 2021. Now I need to gather information about Uber's revenue growth in 2021 to compare and contrast the two.
Action: uber_10k
Action Input: {'input': 'revenue growth in 2021'}
Observation: 
Revenue growth in 2021 was 57%, with total revenue increasing from $11.1 billion in 2020 to $17.5 billion in 2021.
Response: In 2021, Lyft's revenue grew by 36%, while Uber's revenue grew by 57%. This indicates that Uber experienced a higher revenue growth rate compared to Lyft. Additionally, Uber's total revenue increased from $11.1 billion in 2020 to $17.5 billion in 2021, while Lyft's specific revenue figures are not available. Therefore, Uber had a higher revenue gr